## 5.5.1 클래스 선언을 위한 준비

In [2]:
%run ../chap05/mathutil.ipynb

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

Exception: File `'../chap05/mathutil.ipynb.py'` not found.

## 5.5.2 Model 클래스의 선언

In [3]:
class Model(object):
    def __init__(self, name, dataset):
        self.name = name
        self.dataset = dataset
        self.is_training = False
        if not hasattr(self, 'rand_std'): self.rand_std = 0.030 # hasattr: 선택된 객채 내에 변수 존재 여부 확인
            
    def __str__(self):
        return '{}/{}'.format(self.name, self.dataset)
    
    def exec_all(self, epoch_count = 10, batch_size = 10, learning_rate = 0.001, 
                report = 0, show_cnt = 3):
        self.train(epoch_count, batch_size, learning_rate, report)
        self.test()
        if show_cnt > 0: self.visualize(show_cnt)

## 5.5.3 MlpModel 클래스의 선언

In [4]:
class MlpModel(Model):
    def __init__(self, name, dataset, hconfigs):
        super(MlpModel, self).__init__(name, dataset)
        self.init_parameters(hconfigs)

## 5.5.4 파라미터 생성 메서드의 정의

In [5]:
def mlp_init_parameters(self, hconfigs):
    self.hconfigs = hconfigs
    self.pm_hiddens = []
    
    prev_shape = self.dataset.input_shape # 입력 벡터 크기 정보는 전역 변수가 아니라 dataset 객체의 속성값으로부터 얻음
    
    for hconfig in hconfigs:
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig)
        self.pm_hiddens.append(pm_hidden) # 생성된 파라미터들을 전역 변수에 저장하는 대신 객체 변수로 저장한다.
        
    output_cnt = int(np.prod(self.dataset.output_shape)) # 출력 벡터 크기의 정보도 전역 변수가 아니라 dataset 객체의 속성값으로부터 얻음
                                                         # 파라미터 생성에 반영되는 벡터 크기를 np.prod() 함수를 이용해 변환함
                                                         # 앞으로 입출력 벡터나 은닉 벡터 크기가 자연수가 아닌 다차원 크기일 수 있기 때문
    self.pm_output, _ = self.alloc_layer_param(prev_shape, output_cnt) 
    
def mlp_alloc_layer_param(self, input_shape, hconfig):
    input_cnt = np.prod(input_shape)
    output_cnt = hconfig
    
    weight, bias = self.alloc_params_pair([input_cnt, output_cnt])
    
    return {'w':weight, 'b':bias}, output_cnt

def mlp_alloc_param_pair(self, shape):
    weight = np.random.normal(0, self.rand_std, shape)
    bias = np.zeros([shape[-1]])
    return weight, bias

MlpModel.init_parameters = mlp_init_parameters
MlpModel.alloc_layer_param = mlp_alloc_layer_param
MlpModel.alloc_param_pair = mlp_alloc_param_pair

## 5.5.5 학습 메서드의 정의

In [6]:
def mlp_model_train(self, epoch_count = 10, batch_size = 10,
                   learning_rate = 0.001, report = 0):
    self.learning_rate = learning_rate
    
    batch_count = int(self.dataset.train_count / batch_size)
    time1 = time2 = int(time.time())
    if report != 0:
        print('Model {} train started:'.format(self.name))
        
    for epoch in range(epoch_count):
        costs = []
        accs = []
        self.dataset.shuffle_train_data(batch_size * batch_count)
        for n in range(batch_count):
            trX, trY = self.dataset.get_train_data(batch_size, n)
            cost, acc = self.train_step(trX, trY)
            costs.append(cost)
            accs.append(acc)
        
        if report > 0 and (epoch + 1) % report == 0:
            vaX, vaY = self.dataset.get_validate_data(100)
            acc = self.eval_accuaracy(vaX, vaY)
            time3 = int(time.time())
            tm1, tm2 = time3 - time2, time3 - time1
            self.dataset.train_prt_result(epoch+1, costs, accs, acc, tm1, tm2)
            time2 = time3
            
        tm_total = int(time.time()) - time1
        print('Model {} train ended in {} secs:'.format(self.name, tm_total))
MlpModel.train = mlp_model_train

## 5.5.6 평가 메서드의 정의

In [8]:
def mlp_model_test(self):
    teX, teY = self.dataset.get_test_data()
    time1 = int(time.time())
    acc = self.eval_accuracy(teX, teY)
    time2 = int(time.time())
    self.dataset.test_prt_result(self.name, acc, time2-time1)
    
MlpModel.test = mlp_model_test

## 5.5.7 시각화 메서드의 정의

In [10]:
def mlp_model_visualize(self, num):
    print('Model {} Visualization'.format(self.name))
    deX, deY = self.dataset.get_visualize_data(num)
    est = self.get_estimate(deX)
    self.dataset.visualize(deX, deY)
    
MlpModel.visualize = mlp_model_visualize

## 5.5.8 미니배치 학습 메서드의 정의

In [11]:
def mlp_train_step(self, x, y):
    self.is_training = True
    
    output, aux_nn = self.forward_neuralnet(x)
    loss, aux_pp = self.forward_postproc(output, y)
    accuracy = self.eval_accuracy(x, y, output)
    
    G_loss = 1.0
    G_output = self.backprop_postproc(G_loss, aux_pp)
    self.backprop_neuralnet(G_output, aux_nn)
    
    self.is_training = False
    
    return loss, accuracy

MlpModel.train_step = mlp_train_step

# 5.5.9 신경망 부분에 대한 순전파 및 역전파 메서드 정의

In [12]:
def mlp_forward_neuralnet(self, x):
    hidden = x
    aux_layers = []
    
    for n, hconfig in enumerate(self.hconfigs):
        hidden, aux = self.forward_layer(hidden, hconfig, self.pm_hiddens[n])
        aux_layers.append(aux)
        
    output, aux_out = self.forward_layer(hidden, None, self.pm_output)
    
    return output, [aux_out, aux_layers]

def mlp_backprop_neuralnet(self, G_output, aux):
    aux_out, aux_layers = aux
    
    G_hidden = self.backprop_layer(G_output, None, self.pm_output, aux_out)
    
    for n in reversed(range(len(self.hconfigs))):
        hconfig, pm, aux = self.hconfigs[n], self.pm_hidens[n], aux_layers[n]
        G_hidden = self.backprop_layer(G_hidden, hconfig, pm, aux)
        
    return G_hidden

MlpModel.forward_neuralnet = mlp_forward_neuralnet
MlpModel.backprop_neuralnet = mlp_backprop_neuralnet

# 5.5.10 계층 하나에 대한 순전파 및 역전파 메서드 정의